In [ ]:
import os
import itertools

# From python recipes page on itertools
def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return itertools.zip_longest(*args, fillvalue=fillvalue)

def get_contents(fn):
    with open(fn, 'r') as f:
        stuff = f.read().splitlines()
    data = [s.split(",") for s in stuff]
    return csvify(data)

def csv_eof(s):
    return s is None or s == "Start Time" or s == "Batch Name"

def iter_data(data):
    for t, pv, _ in grouper(data, 3, (None, None, None)):
        if csv_eof(t[0]):
            break
        yield t, pv

def remove_unneeded(data):
    result = []
    for t, pv in iter_data(data):
        t = list(t); pv = list(pv)
        t[0] += ".Time"
        pv[0] += ".PV"
        result.append(t)
        result.append(pv)
    return result

def csvify(data):
    return list(zip(*data))

def write_data(fn, data):
    txt = "\n".join(",".join(r) for r in data)
    with open(fn, 'w') as f:
        f.write(txt)
        
def condense(contents):
    data = remove_unneeded(contents)
    return csvify(data)

def condense_file(file):
    contents = get_contents(file)
    return condense(contents)
        
def make_condensed_file(oldfile, newfile):
    csvd = condense_file(oldfile)
    write_data(newfile, csvd)

In [ ]:
fn = "csvfix_test_semma.csv"
make_condensed_file(fn, "csvfix_test_output.csv")

In [16]:
from dateutil.parser import parse as parse_date

class Data():
    def __init__(self, hd, lheaders):
        self._headers = hd.copy()
        self._lheaders = lheaders
        self._data = {}
        self._nonerow = [""] * len(lheaders)
        
    def add(self, ts, var, value):
        if ts not in self._data:
            vals = self._data[ts] = self._nonerow.copy()
        else:
            vals = self._data[ts]
        idx = self._headers[var]
        if vals[idx] != "":
            raise ValueError("Duplicate value for: %r %r: %r"% (ts, var, value))
        vals[idx] = value     
        
    def get(self):
        return self._data
    
    @property
    def headers(self):
        return self._lheaders.copy()
        

def compile_data(contents):
    hd = {}
    headers = []
    itc = iter(contents)
    for i, (h, _, _) in enumerate(grouper(next(itc), 3, (None, None, None))):
        if csv_eof(h):
            break
        hd[h] = i
        headers.append(h)
        
    # Scan the data row by row, using the generated header index
    # to determine which slot in the list to insert the given value.
    # If the timestamp is empty, that indicates there is no data to 
    # record, so skip and continue. 
        
    _parse_date = parse_date
    alld = Data(hd, headers)
    for row in itc:
        for h, (t, pv, _) in zip(headers, grouper(row, 3)):
            if not t:
                continue  # no more data for this variable
            td = _parse_date(t)
            alld.add(td, h, pv)
    return alld

        
def get_contents2(file):
    with open(file, 'r') as f:
        stuff = f.read().splitlines()
    res = []
    for row in stuff:
        s = row.strip()
    
        if s:
            res.append(s.split(","))
    return res

def sparse2csv(alld):
    sdata = sorted(alld.get().items(), key=lambda t: t[0])
    
    o = ['Timestamp', 'Elapsed(hr)']
    o.extend(alld.headers)
    out = [o]
    t1 = sdata[0][0]
    for ts, values in sdata:
        t = ts.strftime("%m/%d/%Y %I:%M:%S %p")
        o = [t, str((ts-t1).total_seconds()/3600)]
        o.extend(values)
        out.append(o)
    return out

def sparsify(contents):
    data = compile_data(contents)
    return sparse2csv(data)

def make_sparse_file(oldfile, newfile):
    contents = get_contents2(oldfile)
    csvd = sparsify(contents)
    write_data(newfile, csvd)

In [18]:
make_sparse_file("csvfix_test2_input.csv", "csvfix_sparse_output.csv")